ERROR: Can't invoke 'meta.llama3-70b-instruct-v1:0'. Reason: An error occurred (InvalidSignatureException) when calling the InvokeModel operation: The request signature we calculated does not match the signature you provided. Check your AWS Secret Access Key and signing method. Consult the service documentation for details.


In [ ]:
!pip install --upgrade boto3
!pip install python-dotenv

In [2]:
import os
import json
import boto3
from botocore.exceptions import ClientError
import re
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

class BedrockClient:
    def __init__(self, region_name, model_id):
        self.client = boto3.client('bedrock-runtime', region_name=region_name)
        self.model_id = model_id
        self.dataset = self.load_dataset()[:5]  # Load only first 5 questions

    def load_dataset(self):
        with open('RepresentativeDataset.json', 'r') as f:
            return json.load(f)

    def create_prompt(self, question, options):
        prompt = f"Solve the following mathematical problem:\n\nQuestion: {question}\n"
        prompt += f"Options: {', '.join(options)}\n\n"
        prompt += "Provide your solution and clearly state your final answer at the end, prefixed with the option choice as an alphabet for eg ('a' if 1 ) as 'Final Answer: '."

        return f"""<begin_of_text>
<start_header_id>user<end_header_id>
{prompt}
<eot_id>
<start_header_id>assistant<end_header_id>
"""

    def invoke_model(self, question, options, max_tokens=1024, temperature=0.7):
        formatted_prompt = self.create_prompt(question, options)

        request_body = {
            "prompt": formatted_prompt,
            "max_gen_len": max_tokens,
            "temperature": temperature,
        }

        try:
            response = self.client.invoke_model(
                body=json.dumps(request_body),
                contentType='application/json',
                modelId=self.model_id
            )
            response_body = json.loads(response['body'].read().decode('utf-8'))
            return response_body.get('generation', '')
        except ClientError as e:
            print(f"ERROR: Can't invoke '{self.model_id}'. Reason: {e}")
            return None

    def extract_model_answer(self, model_solution):
        match = re.search(r"Final Answer:\s*(\w+)", model_solution, re.IGNORECASE)
        if match:
            return match.group(1).lower()
        return None

    def process_questions(self):
        results = []
        correct_count = 0
        for problem in self.dataset:
            result = self.invoke_model(problem['question'], problem['options'])
            model_answer = self.extract_model_answer(result)
            is_correct = model_answer == problem['answer'].lower() if model_answer else False
            if is_correct:
                correct_count += 1

            results.append({
                'question': problem['question'],
                'model_solution': result,
                'model_answer': model_answer,
                'correct_solution': problem['solution'],
                'correct_answer': problem['answer'],
                'is_correct': is_correct
            })

        accuracy = correct_count / len(self.dataset) if self.dataset else 0
        return results, accuracy

def main():
    region_name = os.environ.get('AWS_REGION', 'us-east-1')
    model_id = os.environ.get('BEDROCK_MODEL_ID', 'meta.llama2-70b-chat-v1')

    client = BedrockClient(region_name, model_id)
    results, accuracy = client.process_questions()

    # Print results
    for i, result in enumerate(results, 1):
        print(f"\nQuestion {i}:")
        print(f"Q: {result['question']}")
        print(f"Model's solution:\n{result['model_solution']}")
        print(f"Model's extracted answer: {result['model_answer']}")
        print(f"Correct solution: {result['correct_solution']}")
        print(f"Correct answer: {result['correct_answer']}")
        print(f"Is correct: {result['is_correct']}")
        print("-" * 50)

    print(f"\nOverall Accuracy: {accuracy:.2%}")

if __name__ == "__main__":
    main()


Question 1:
Q: If \(\int\left(e^{2 x}+2 e^{x}-e^{-x}-1\right) e^{\left(e^{x}+e^{-x}\right)} d x=g(x) e^{\left(e^{x}+e^{-x}\right)}+c\), where \(c\) is a constant of integeration, then \(g(0)\) is equal to:
Model's solution:
Let's solve the given integral:

$$\int\left(e^{2x}+2e^x-e^{-x}-1\right)e^{\left(e^x+e^{-x}\right)}dx=g(x)e^{\left(e^x+e^{-x}\right)}+c$$

To evaluate this integral, we can use the substitution method. Let's substitute:

$$u=e^x+e^{-x}$$

Then, we can find the derivative of \(u\) with respect to \(x\):

$$\frac{du}{dx}=e^x-e^{-x}$$

Now, we can rewrite the integral as:

$$\int\left(e^{2x}+2e^x-e^{-x}-1\right)e^u du=g(u)e^u+c$$

Notice that the integral on the left-hand side is now a product of two functions, which can be integrated using the product rule:

$$\int\left(e^{2x}+2e^x-e^{-x}-1\right)e^u du=e^u\left(e^x+e^{-x}\right)+c$$

Comparing this result with the given equation, we can see that:

$$g(x)=e^x+e^{-x}$$

Now, we need to find the value of \(g(0)\). Subs

In [3]:
import random
import re

def pair_sentences(solution):
    """
    Break the solution into pairs of consecutive sentences.

    Args:
        solution (str): The mathematical solution.

    Returns:
        list of tuple: A list of pairs of consecutive sentences.
    """
    # Split the solution into individual sentences
    sentences = solution.split('\n')

    # Create pairs of consecutive sentences
    sentence_pairs = [(sentences[i], sentences[i + 1]) for i in range(len(sentences) - 1)]

    return sentence_pairs

def apply_all_transformations(sentence):
    """
    Apply all the transformation functions to the sentence.

    Args:
        sentence (str): The correct mathematical sentence.

    Returns:
        list of str: A list of sentences with different errors applied.
    """
    error_functions = [change_constants, swap_operators, change_variables, incorrect_simplification]

    # Apply all transformation functions to the sentence
    transformed_sentences = []
    for func in error_functions:
        transformed_sentences.append(func(sentence))

    return transformed_sentences

def change_constants(sentence):
    """
    Randomly change constants or coefficients in the mathematical sentence.
    """
    return re.sub(r'(\d+)', lambda x: str(int(x.group(0)) + random.randint(1, 3)), sentence)

def swap_operators(sentence):
    """
    Randomly swap operators in the mathematical sentence (+ becomes -, * becomes /, etc.).
    """
    operator_map = {'+': '-', '-': '+', '*': '/', '/': '*'}
    return re.sub(r'[+\-*/]', lambda x: operator_map.get(x.group(0), x.group(0)), sentence)

def change_variables(sentence):
    """
    Randomly change variables in the mathematical sentence (e.g., change 'x' to 'y').
    """
    variable_map = {'x': 'y', 'y': 'x', 't': 's', 's': 't'}
    return re.sub(r'[xyts]', lambda x: variable_map.get(x.group(0), x.group(0)), sentence)

def incorrect_simplification(sentence):
    """
    Introduce an incorrect simplification or manipulation in the sentence.
    """
    return re.sub(r'e\^\{(\d* ?x)\}', lambda x: f"e^{{{random.choice(['2x', 'x', '3x'])}}}", sentence)

def create_no_dataset(sentence_pair, n_errors=3):
    """
    Create multiple incorrect versions of the sentence pair for the 'no' dataset.

    Args:
        sentence_pair (tuple): A tuple containing two consecutive sentences (correct).
        n_errors (int): Number of incorrect versions to generate per pair.

    Returns:
        list of tuple: A list of incorrect sentence pairs.
    """
    sentence_1, sentence_2 = sentence_pair

    # Apply all possible transformations to the second sentence
    incorrect_sentences = apply_all_transformations(sentence_2)

    # Select a random subset of transformations to create the 'no' dataset
    no_dataset = []
    for _ in range(n_errors):
        random_incorrect_sentence = random.choice(incorrect_sentences)
        no_dataset.append((sentence_1, random_incorrect_sentence))

    return no_dataset

def create_datasets(solution, n_errors=3):
    """
    Create both 'yes' (correct) and 'no' (incorrect) datasets from the solution.

    Args:
        solution (str): The correct mathematical solution.
        n_errors (int): Number of incorrect pairs to generate per correct pair.

    Returns:
        tuple: A list of correct pairs ('yes' dataset) and a list of incorrect pairs ('no' dataset).
    """
    # Get the pairs of consecutive sentences
    sentence_pairs = pair_sentences(solution)

    # Create the 'yes' dataset (correct pairs)
    yes_dataset = sentence_pairs.copy()

    # Create the 'no' dataset (incorrect pairs)
    no_dataset = []
    for pair in sentence_pairs:
        no_dataset.extend(create_no_dataset(pair, n_errors=n_errors))

    return yes_dataset, no_dataset

# Example Usage
correct_solution = """\\item (d) \\(\\int\\left(e^{2 x}+2 e^{x}-e^{-x}-1\\right) \\cdot e^{\\left(e^{x}+e^{-x}\\right)} d x\\)
\\(I=\\int\\left(e^{2 x}+e^{x}-1\\right) \\cdot e^{\\left(e^{x}+e^{-x}\\right)} d x+\\int\\left(e^{x}-e^{-x}\\right) e^{\\left(e^{x}+e^{-x}\\right)} d x\\)
\\(=\\int e^{x}\\left(e^{x}+1-e^{-x}\\right) \\cdot e^{\\left(e^{x}+e^{-x}\\right)} d x+e^{\\left(e^{x}+e^{-x}\\right)}\\)
\\(=\\int\\left(e^{x}-e^{-x}+1\\right) e^{\\left(e^{x}+e^{-x}+x\\right)} d x+e^{\\left(e^{x}+e^{-x}\\right)}\\)
Let \\(e^{x}+e^{-x}+x=t \\Rightarrow\\left(e^{x}+e^{-x}+1\\right) d x=d t\\)
\\(=\\int e^{t} d t+e^{\\left(e^{x}+e^{-x}\\right)}=e^{t}+e^{\\left(e^{x}+e^{-x}\\right)}+C\\)
\\(=e^{\\left(e^{x}+e^{-x}+x\\right)}+e^{\\left(e^{x}+e^{-x}\\right)}+C\\)
\\(=\\left(e^{x}+1\\right) \\cdot e^{\\left(e^{x}+e^{-x}\\right)}+C\\)
So, \\(g(x)=1+e^{x}\\) and \\(g(0)=2\\)"""

# Generate 'yes' and 'no' datasets, with 5 errors per correct pair
yes_dataset, no_dataset = create_datasets(correct_solution, n_errors=5)

print("Yes Dataset (Correct Pairs):")
for pair in yes_dataset:
    print(pair)

print("\nNo Dataset (Incorrect Pairs):")
for pair in no_dataset:
    print(pair)


Yes Dataset (Correct Pairs):
('\\item (d) \\(\\int\\left(e^{2 x}+2 e^{x}-e^{-x}-1\\right) \\cdot e^{\\left(e^{x}+e^{-x}\\right)} d x\\)', '\\(I=\\int\\left(e^{2 x}+e^{x}-1\\right) \\cdot e^{\\left(e^{x}+e^{-x}\\right)} d x+\\int\\left(e^{x}-e^{-x}\\right) e^{\\left(e^{x}+e^{-x}\\right)} d x\\)')
('\\(I=\\int\\left(e^{2 x}+e^{x}-1\\right) \\cdot e^{\\left(e^{x}+e^{-x}\\right)} d x+\\int\\left(e^{x}-e^{-x}\\right) e^{\\left(e^{x}+e^{-x}\\right)} d x\\)', '\\(=\\int e^{x}\\left(e^{x}+1-e^{-x}\\right) \\cdot e^{\\left(e^{x}+e^{-x}\\right)} d x+e^{\\left(e^{x}+e^{-x}\\right)}\\)')
('\\(=\\int e^{x}\\left(e^{x}+1-e^{-x}\\right) \\cdot e^{\\left(e^{x}+e^{-x}\\right)} d x+e^{\\left(e^{x}+e^{-x}\\right)}\\)', '\\(=\\int\\left(e^{x}-e^{-x}+1\\right) e^{\\left(e^{x}+e^{-x}+x\\right)} d x+e^{\\left(e^{x}+e^{-x}\\right)}\\)')
('\\(=\\int\\left(e^{x}-e^{-x}+1\\right) e^{\\left(e^{x}+e^{-x}+x\\right)} d x+e^{\\left(e^{x}+e^{-x}\\right)}\\)', 'Let \\(e^{x}+e^{-x}+x=t \\Rightarrow\\left(e^{x}+e^{-x}+1\

In [8]:
import json
import random
import re

def change_constants(sentence):
    return re.sub(r'(\d+)', lambda x: str(int(x.group(0)) + random.randint(1, 3)), sentence)

def swap_operators(sentence):
    operator_map = {'+': '-', '-': '+', '*': '/', '/': '*'}
    return re.sub(r'[+\-*/]', lambda x: operator_map.get(x.group(0), x.group(0)), sentence)

def change_variables(sentence):
    variable_map = {'x': 'y', 'y': 'x', 't': 's', 's': 't'}
    return re.sub(r'[xyts]', lambda x: variable_map.get(x.group(0), x.group(0)), sentence)

def incorrect_simplification(sentence):
    return re.sub(r'e\^\{(\d* ?x)\}', lambda x: f"e^{{{random.choice(['2x', 'x', '3x'])}}}", sentence)

def apply_all_transformations(sentence):
    error_functions = [change_constants, swap_operators, change_variables, incorrect_simplification]
    transformed_sentences = []
    for func in error_functions:
        transformed_sentences.append(func(sentence))
    return transformed_sentences

def pair_sentences(solution):
    sentences = [s.strip() for s in solution.split('\n') if s.strip() and len(s.strip()) >= 3]
    sentence_pairs = [(sentences[i], sentences[i + 1]) for i in range(len(sentences) - 1)]
    return sentence_pairs

def create_no_dataset(sentence_pair, n_errors=3):
    sentence_1, sentence_2 = sentence_pair
    incorrect_sentences = apply_all_transformations(sentence_2)
    no_dataset = []
    for _ in range(n_errors):
        random_incorrect_sentence = random.choice(incorrect_sentences)
        if len(sentence_1) >= 3 and len(random_incorrect_sentence) >= 3:
            no_dataset.append((sentence_1, random_incorrect_sentence))
    return no_dataset

def create_datasets(solution, n_errors=3):
    sentence_pairs = pair_sentences(solution)
    yes_dataset = [{"sentence_pair": pair, "ans": "correct"} for pair in sentence_pairs if len(pair[0]) >= 3 and len(pair[1]) >= 3]
    no_dataset = []
    for pair in sentence_pairs:
        incorrect_pairs = create_no_dataset(pair, n_errors=n_errors)
        no_dataset.extend([{"sentence_pair": incorrect_pair, "ans": "incorrect"} for incorrect_pair in incorrect_pairs if len(incorrect_pair[0]) >= 3 and len(incorrect_pair[1]) >= 3])
    return yes_dataset, no_dataset

def process_representative_dataset(filename):
    with open(filename, 'r') as file:
        dataset = json.load(file)

    full_dataset = []
    for question in dataset:
        solution = question["solution"]
        yes_dataset, no_dataset = create_datasets(solution, n_errors=5)
        full_dataset.extend(yes_dataset + no_dataset)

    return full_dataset

input_filename = 'RepresentativeDataset.json'
output_filename = 'processed_dataset3.json'

full_dataset = process_representative_dataset(input_filename)

with open(output_filename, 'w') as json_file:
    json.dump(full_dataset, json_file, indent=4)

print(f"Processed dataset has been exported to '{output_filename}'.")
print("Sample of processed dataset:")
for item in full_dataset[:50]:
    print(item)


Processed dataset has been exported to 'processed_dataset3.json'.
Sample of processed dataset:
{'sentence_pair': ('\\item (d) \\(\\int\\left(e^{2 x}+2 e^{x}-e^{-x}-1\\right) \\cdot e^{\\left(e^{x}+e^{-x}\\right)} d x\\)', '\\(I=\\int\\left(e^{2 x}+e^{x}-1\\right) \\cdot e^{\\left(e^{x}+e^{-x}\\right)} d x+\\int\\left(e^{x}-e^{-x}\\right) e^{\\left(e^{x}+e^{-x}\\right)} d x\\)'), 'ans': 'correct'}
{'sentence_pair': ('\\(I=\\int\\left(e^{2 x}+e^{x}-1\\right) \\cdot e^{\\left(e^{x}+e^{-x}\\right)} d x+\\int\\left(e^{x}-e^{-x}\\right) e^{\\left(e^{x}+e^{-x}\\right)} d x\\)', '\\(=\\int e^{x}\\left(e^{x}+1-e^{-x}\\right) \\cdot e^{\\left(e^{x}+e^{-x}\\right)} d x+e^{\\left(e^{x}+e^{-x}\\right)}\\)'), 'ans': 'correct'}
{'sentence_pair': ('\\(=\\int e^{x}\\left(e^{x}+1-e^{-x}\\right) \\cdot e^{\\left(e^{x}+e^{-x}\\right)} d x+e^{\\left(e^{x}+e^{-x}\\right)}\\)', '\\(=\\int\\left(e^{x}-e^{-x}+1\\right) e^{\\left(e^{x}+e^{-x}+x\\right)} d x+e^{\\left(e^{x}+e^{-x}\\right)}\\)'), 'ans': 'correct'